In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3588736 sha256=a09cd5351c6e85a8b29abdecddb34cf07c847de74f4d6147b69a4caaeb4c2650
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=2731966 sha256=262072816cc9ff7f0c39c63c470e4b32d791a7c888e3e3c10833086e6c998dd9
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch-sparse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

In [ ]:
# Load data
users_df = pd.read_csv('/content/drive/MyDrive/data/users_neighborhood_anon.csv')
edges_df = pd.read_csv('/content/drive/MyDrive/data/users.edges')
G = nx.read_graphml('/content/drive/MyDrive/data/users_clean.graphml')

In [ ]:
users_df.head()

,user_id,hate,hate_neigh,normal_neigh,statuses_count,followers_count,followees_count,favorites_count,listed_count,betweenness,...,c_feminine_empath,c_medieval_empath,c_journalism_empath,c_farming_empath,c_plant_empath,c_shopping_empath,c_ship_empath,c_religion_empath,c_tourism_empath,c_power_empath
0,0,normal,True,True,101767,3504,3673,81635,53,100467.895084,...,0.001380,0.003288,0.000255,0.002189,0.000593,0.003689,0.003559,0.001634,0.002662,0.007487
1,1,other,False,False,2352,19609,309,61,197,0.000000,...,0.000802,0.004465,0.000444,0.001632,0.001298,0.002183,0.008969,0.004975,0.000647,0.003419
2,2,other,False,False,1044,2371,2246,561,16,4897.117853,...,0.000985,0.005284,0.000554,0.001084,0.001359,0.002041,0.016846,0.004881,0.001214,0.003347
3,3,other,False,False,167172,3004,298,3242,53,9.864754,...,0.001391,0.002061,0.001116,0.001282,0.000000,0.001951,0.015423,0.000446,0.000446,0.005241
4,4,other,False,False,1998,17643,19355,485,239,0.000000,...,0.000633,0.035382,0.000317,0.000475,0.000475,0.002431,0.007656,0.033273,0.072792,0.003698


In [ ]:
print(users_df.shape)
users_df_copy = users_df

(100386, 1039)


In [ ]:
users_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100386 entries, 0 to 100385
Columns: 1039 entries, user_id to c_power_empath
dtypes: bool(6), float64(1025), int64(6), object(2)
memory usage: 791.7+ MB


In [ ]:
users_df_copy.isnull().sum()

user_id                 0
hate                    0
hate_neigh              0
normal_neigh            0
statuses_count          0
                     ... 
c_shopping_empath    4008
c_ship_empath        4008
c_religion_empath    4008
c_tourism_empath     4008
c_power_empath       4008
Length: 1039, dtype: int64

In [ ]:
# Drop rows with missing values
users_df_copy.dropna(inplace=True)
users_df_copy.head(5)

,user_id,hate,hate_neigh,normal_neigh,statuses_count,followers_count,followees_count,favorites_count,listed_count,betweenness,...,c_feminine_empath,c_medieval_empath,c_journalism_empath,c_farming_empath,c_plant_empath,c_shopping_empath,c_ship_empath,c_religion_empath,c_tourism_empath,c_power_empath
0,0,normal,True,True,101767,3504,3673,81635,53,100467.895084,...,0.001380,0.003288,0.000255,0.002189,0.000593,0.003689,0.003559,0.001634,0.002662,0.007487
1,1,other,False,False,2352,19609,309,61,197,0.000000,...,0.000802,0.004465,0.000444,0.001632,0.001298,0.002183,0.008969,0.004975,0.000647,0.003419
2,2,other,False,False,1044,2371,2246,561,16,4897.117853,...,0.000985,0.005284,0.000554,0.001084,0.001359,0.002041,0.016846,0.004881,0.001214,0.003347
3,3,other,False,False,167172,3004,298,3242,53,9.864754,...,0.001391,0.002061,0.001116,0.001282,0.000000,0.001951,0.015423,0.000446,0.000446,0.005241
4,4,other,False,False,1998,17643,19355,485,239,0.000000,...,0.000633,0.035382,0.000317,0.000475,0.000475,0.002431,0.007656,0.033273,0.072792,0.003698


In [ ]:
users_df_copy.isnull().sum()

user_id              0
hate                 0
hate_neigh           0
normal_neigh         0
statuses_count       0
                    ..
c_shopping_empath    0
c_ship_empath        0
c_religion_empath    0
c_tourism_empath     0
c_power_empath       0
Length: 1039, dtype: int64

In [ ]:
chunk_size = 100
for i in range(0, len(users_df_copy.columns), chunk_size):
    print(users_df_copy.columns[i:i+chunk_size])

Index(['user_id', 'hate', 'hate_neigh', 'normal_neigh', 'statuses_count',
       'followers_count', 'followees_count', 'favorites_count', 'listed_count',
       'betweenness', 'eigenvector', 'in_degree', 'out_degree',
       'negotiate_empath', 'vehicle_empath', 'science_empath',
       'timidity_empath', 'gain_empath', 'swearing_terms_empath',
       'office_empath', 'tourism_empath', 'computer_empath', 'violence_empath',
       'horror_empath', 'dispute_empath', 'sadness_empath', 'internet_empath',
       'college_empath', 'attractive_empath', 'plant_empath',
       'technology_empath', 'reading_empath', 'deception_empath',
       'ancient_empath', 'ship_empath', 'messaging_empath', 'surprise_empath',
       'occupation_empath', 'hearing_empath', 'furniture_empath',
       'restaurant_empath', 'domestic_work_empath', 'art_empath', 'pet_empath',
       'air_travel_empath', 'dance_empath', 'healing_empath', 'smell_empath',
       'sympathy_empath', 'anger_empath', 'neglect_empath', 'fa

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelEncoder

In [ ]:

sample_size = 90000
sample_indices = pd.Series(users_df_copy.index).sample(n=sample_size, random_state=42)
sample_df = users_df.loc[sample_indices]

# Separate features (X) and target variable (y)
X = sample_df.drop(['user_id', 'hate'], axis=1)  # Assuming 'user_id' and 'hate' are not features
y = sample_df['hate']

# Remove non-numeric features
numeric_features = X.select_dtypes(include=[np.number]).columns
X = X[numeric_features]

# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Create a Lasso regression model
lasso = Lasso(alpha=0.1, random_state=42)  # Adjust the alpha parameter as needed

# Fit the Lasso model to the data
lasso.fit(X, y)

# Get the feature coefficients
coefficients = lasso.coef_

# Get the selected features (non-zero coefficients)
selected_features = X.columns[np.abs(coefficients) > 0]  # Selecting all non-zero coefficients

# If the number of selected features is less than 20, sort the coefficients and select the top 20
if len(selected_features) < 20:
    sorted_coefficients = np.abs(coefficients).argsort()[::-1]
    selected_features = X.columns[sorted_coefficients[:20]]

# Create a new DataFrame with selected features
selected_features_df = X[selected_features]

# Print the selected features
print("Selected features:")
print(selected_features)

Selected features:
Index(['statuses_count', 'followers_count', 'followees_count',
       'favorites_count', 'listed_count', 'betweenness', 'retweet number',
       'baddies', 'mentions', 'time_diff', 'time_diff_median', 'created_at',
       'c_statuses_count', 'c_followers_count', 'c_followees_count',
       'c_favorites_count', 'c_listed_count', 'c_time_diff',
       'c_time_diff_median', 'c_betweenness', 'c_number hashtags',
       'c_tweet number', 'c_baddies', 'c_mentions'],
      dtype='object')


### **The END**